<a href="https://colab.research.google.com/github/xingshuyin/lhy/blob/master/hugging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets

In [2]:
from transformers import BertTokenizer, BertModel, AdamW, TrainingArguments, Trainer, AutoModelForSequenceClassification
from datasets import load_dataset
import torch.nn as nn
import torch

In [ ]:
dataset = load_dataset('lansinuote/ChnSentiCorp')

In [ ]:
token = BertTokenizer.from_pretrained('bert-base-chinese')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-chinese')
model.to('cuda')

In [6]:
data_train = dataset['train']
data_validate = dataset['validation']
data_test = dataset['test']

In [10]:
def tokenize(batch):
    return token(batch['text'], padding=True, truncation=True)
data_train = data_train.map(tokenize, batched=True, batch_size=len(data_train))
data_validate = data_validate.map(tokenize, batched=True, batch_size=len(data_validate))
data_test = data_test.map(tokenize, batched=True, batch_size=len(data_test))

Map:   0%|          | 0/9600 [00:00<?, ? examples/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [12]:
data_train.set_format('torch', columns=data_train.features)
data_validate.set_format('torch', columns=data_validate.features)
data_test.set_format('torch', columns=data_test.features)

In [13]:
trainer_args = TrainingArguments("./result", evaluation_strategy='epoch', learning_rate=2e-5, per_device_train_batch_size=16,per_device_eval_batch_size=16, num_train_epochs=1,weight_decay=0.01)
trainer = Trainer(model=model, args=trainer_args, train_dataset=data_train, eval_dataset=data_validate)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


In [21]:
trainer.predict(data_test)

PredictionOutput(predictions=array([[ 0.7767172 , -0.64956784],
       [ 1.5641962 , -1.4242151 ],
       [ 2.2087724 , -1.8000623 ],
       ...,
       [-1.9116296 ,  3.2708416 ],
       [-1.419592  ,  2.765947  ],
       [ 2.47258   , -2.2697482 ]], dtype=float32), label_ids=None, metrics={'test_runtime': 38.7192, 'test_samples_per_second': 30.992, 'test_steps_per_second': 1.937})

In [25]:
dataset['test'][1:5]

{'text': ['怀着十分激动的心情放映，可是看着看着发现，在放映完毕后，出现一集米老鼠的动画片！开始还怀疑是不是赠送的个别现象，可是后来发现每张DVD后面都有！真不知道生产商怎么想的，我想看的是猫和老鼠，不是米老鼠！如果厂家是想赠送的话，那就全套米老鼠和唐老鸭都赠送，只在每张DVD后面添加一集算什么？？简直是画蛇添足！！',
  '还稍微重了点，可能是硬盘大的原故，还要再轻半斤就好了。其他要进一步验证。贴的几种膜气泡较多，用不了多久就要更换了，屏幕膜稍好点，但比没有要强多了。建议配赠几张膜让用用户自己贴。',
  '交通方便；环境很好；服务态度很好 房间较小',
  '不错，作者的观点很颠覆目前中国父母的教育方式，其实古人们对于教育已经有了很系统的体系了，可是现在的父母以及祖父母们更多的娇惯纵容孩子，放眼看去自私的孩子是大多数，父母觉得自己的孩子在外面只要不吃亏就是好事，完全把古人几千年总结的教育古训抛在的九霄云外。所以推荐准妈妈们可以在等待宝宝降临的时候，好好学习一下，怎么把孩子教育成一个有爱心、有责任心、宽容、大度的人。'],
 'label': [0, 0, 1, 1]}